# ChatGPT Mental Support simulation

In this notebook we will simulate how chatGPT will respond to request from people with mental issues, problems and worries, that will be expressed plain in conversation.
This responses will be compared to responses from mental counselor to see how is chatGPT doing this.

### Dataset

This simulation is based on this dataset:

[Mental health Counseling Conversations dataset](https://www.kaggle.com/datasets/thedevastator/nlp-mental-health-conversations)

### Dataset structure

| Column   | Description | 
|----------|----------|
| Context  | Request, phrase, query sentence from the patient, that seems to have mental troubles, problem, disorders of different kind        |  
| Response | Response, that mental counselor is supposed to give. For one ```Context``` there is from 1 to 47 different responses (mean 2.5) | 


### Comparison

Comparison between ChatGPT response will be done with special prompt, resulting in score [0-100], and be done comparing one response from ChatGRT to all (one by one) responses from the dataset. New dataset will chatGPT responses and scores will be build, and then explored



# Notebook for simulation

Exploration of data and cleaning,
Probes with ChatGPT are places in the [preparation notebook](notebook_preparation_work.ipynb)

Here only functions to make data ready, and helper functions are presented

## Preparing the data

The dataset on the link above is available to download, so here it is already present in the ```dataset/NLP_Mental_Health_Conversations.csv```

In [1]:
#!pip install pandas

In [2]:
import pandas as pd
import numpy as np

C:\Users\sirius\AppData\Local\Temp\ipykernel_20852\2162656668.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
CONVERSATIONS_DATASET_FILENAME = 'dataset/NLP_Mental_Health_Conversations.csv'

conversation_dataset = pd.read_csv(CONVERSATIONS_DATASET_FILENAME)

In [4]:
conversation_dataset.columns = conversation_dataset.columns.str.lower().str.replace(' ', '_')

### Removing duplicates and zero entries

In [5]:
conversation_dataset.dropna(subset=['response'], inplace=True)
conversation_dataset.drop_duplicates(subset=['response'], inplace=True)
conversation_dataset.reset_index(drop=True, inplace=True)

So there is duplicate entries, but they differ in heading or trailing spaces.
Let's fix this, and remove duplicates once more.

In [6]:
conversation_dataset['response'] = conversation_dataset['response'].apply(
    lambda x: x.strip())

In [7]:
conversation_dataset.drop_duplicates(subset=['response'], inplace=True)
conversation_dataset.reset_index(drop=True, inplace=True)

### Errors in dataset

Working with the dataset I found some irrelevant entries. So we should just delete them


In [8]:
error_context = [" such as not enough sleep",
                 "Can a counselor take sides with one parent and allow a parent to order the child to tell the counselor"]

conversation_dataset = conversation_dataset[~conversation_dataset['context'].str.startswith(
    tuple(error_context))]

In [9]:
conversation_dataset.describe()

,context,response
count,2049,2049
unique,841,2049
top,I have so many issues to address. I have a his...,"If everyone thinks you're worthless, then mayb..."
freq,47,1


In [10]:
context_unique = conversation_dataset.groupby(
    'context').size().reset_index(name='count')

In [11]:
context_unique.sort_values('count', ascending=False).head(30)

,context,count
286,I have so many issues to address. I have a his...,47
78,How do you know you have the right therapist f...,41
545,"Is it normal for people to cry during therapy,...",38
523,I've gone to a couple therapy sessions so far ...,26
828,When my daughter is stressed about a silly thi...,23
461,I'm going through some things with my feelings...,23
80,How does a person start the counseling process?,22
79,How does a counselor decide when to end counse...,20
31,Does counseling really do anything that can he...,20
73,How do I decide if I should end my relationshi...,15


In [12]:
context_unique['count'].describe()

count    841.000000
mean       2.436385
std        3.538705
min        1.000000
25%        1.000000
50%        1.000000
75%        2.000000
max       47.000000
Name: count, dtype: float64

In [13]:
context_unique['count'].value_counts()

count
1     452
2     182
3      78
4      40
5      23
6      18
9      12
7      10
8       6
11      5
23      2
20      2
15      2
10      2
38      1
26      1
41      1
12      1
47      1
22      1
13      1
Name: count, dtype: int64

In [14]:
CONTEXT_UNIQUE_FILENAME = 'working_data/context_unique.csv'

context_unique.to_csv(CONTEXT_UNIQUE_FILENAME)

### Data is cleaned and ready for simulation

So we have 845 unique requests, with a mean of 2.45 responses for each, and 452 having one response, and max of 47 responses, but only 10 requests have more then 10 responses.

### Conclusions

**Maybe for simulation we should use only requests with 2 or more responses, so we have some variety in approved responses to compare with. If we take request with 2 or more we will have 393 requests, if we take with 3 or more - 209 requests. 209 may be enough for our simulation. And having only one response will make it possible for incorrect scoring of ChatGPT output. Because mental issues is delicate matter, so several different responses may serve well for specific case.**


## ChatGPT

We follow the standard way of using OpenAI API for reaching ChatGPT
The below code uses ```.env``` file in this directory, which is not included to the repository.
```.env```  which must contain:
```
OPENAI_API_KEY=...
```
This API key to be used to interact with OpenAI API

If you want to use this code, you must [get your own API key from OpenAI](https://platform.openai.com/api-keys)

Moreover you will need to make a payment to your ChatGPT account, to be able to use it.

This code is based on the [course by DeepLearning.AI](https://learn.deeplearning.ai/courses/langchain/lesson/2/models%2C-prompts-and-parsers)

In [15]:
# !pip install python-dotenv
# !pip install openai

In [16]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())  # read local .env file

## ChatGPT client and initial functions

In [17]:
GPT_MODEL = 'gpt-3.5-turbo-0125'
#
# Pricing
# Model	Input	Output
# gpt-3.5-turbo-0125	$0.0005 / 1K tokens	$0.0015 / 1K tokens
#
# Updated GPT 3.5 Turbo
# The latest GPT-3.5 Turbo model with higher accuracy at responding in requested formats
# and a fix for a bug which caused a text encoding issue for non-English language function calls.
# Returns a maximum of 4,096 output tokens.
#

In [18]:
client = openai.OpenAI(

    api_key=os.environ.get("OPENAI_API_KEY"),
)

In [19]:
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Hello, this is a test",
        }
    ],
    model=GPT_MODEL,
)

# print(chat_completion.choices[0].message.content)

In [90]:
import random

# default temperature for ChatGPT. Web ChatGPT uses 0.6 all the time. varies from 0 (same answers, no randomness) to 1 (much variety, much random)
TEMPERATURE = 0.6
SPEAK_TEMPERATURE = 0.6

# Our randomness constants:

USE_STARTUP_DIALOG = 0.25

# if we use startup dialog, we may have up to 4 preceding phrases.

# So len 1 will occur more frequently, len 4 - much less
DIALOG_LENGTH_WEIGHTS = [0.35, 0.3, 0.2, 0.15]
DIALOG_LENGTH_OPTIONS = [2, 4, 6, 8]

starting_messages = [
    [
        {'role': 'user', 'content': "Hi"},
        {'role': 'assistant', 'content': "Hello! How can I assist you today?"},
        {'role': 'user', 'content': "Yes! How are you?"},
        {'role': 'assistant', 'content': "As an AI, I don't have feelings or emotions, but I'm here and ready to assist you with any questions or tasks you have. How can I help you today?"},
        {'role': 'user', 'content': "What do you do"},
        {'role': 'assistant', 'content': "I'm here to help with a wide range of tasks! You can ask me questions, seek information on various topics, brainstorm ideas, get assistance with writing or editing, play text-based games, generate creative content, and much more. Just let me know what you need help with, and I'll do my best to assist you!"},
        {'role': 'user', 'content': 'Yes'},
        {'role': 'assistant', 'content': "Great! If you have any specific questions or need assistance with something, feel free to ask. I'm here to help!"},
    ],
    [
        {'role': 'user', 'content': "Hello!"},
        {'role': 'assistant', 'content': "Hello! How can I assist you today?"},
        {'role': 'user', 'content': "I don't know"},
        {'role': 'assistant', 'content': "That's perfectly fine! If you have any questions, need advice, want to discuss a topic, or just want to chat, feel free to let me know. I'm here to help with anything you need."},
        {'role': 'user', 'content': "Ok. I will think about it"},
        {'role': 'assistant', 'content': "Sure thing! Take your time to think, and whenever you're ready, feel free to reach out. I'll be here to assist you whenever you need."},
        {'role': 'user', 'content': "Tell me something"},
        {'role': 'assistant',
            'content': "Sure! Did you know that the world's largest desert isn't the Sahara, but actually Antarctica? Despite being covered in ice, Antarctica is considered a desert because it receives very little precipitation, less than 2 inches (50 millimeters) per year in some areas. Its size and harsh climate make it the largest desert on Earth, even larger than the Sahara Desert."},
    ],
    [
        {'role': 'user', 'content': "Nice to see you"},
        {'role': 'assistant', 'content': "Nice to see you too! How can I assist you today?"},
        {'role': 'user', 'content': "What can you say?"},
        {'role': 'assistant', 'content': "As an AI language model, I can engage in conversation, answer questions, provide information on a wide range of topics, generate text based on prompts, offer suggestions, and more. Just let me know what you need assistance with or what you'd like to discuss!"},
        {'role': 'user', 'content': "Just tell me something"},
        {'role': 'assistant', 'content': "Sure! Did you know that honey never spoils? Archaeologists have found pots of honey in ancient Egyptian tombs that are over 3,000 years old and still perfectly edible. This remarkable longevity is due to the unique chemical composition of honey, which includes low water content and natural acidity, making it inhospitable to bacteria and microorganisms that cause food to spoil."},
        {'role': 'user', 'content': "Interesting"},
        {'role': 'assistant', 'content': "I'm glad you found it interesting! If you have any other topics you'd like to learn about or discuss, feel free to let me know. I'm here to help!"},
    ],
    [
        {'role': 'user', 'content': "How are things"},
        {'role': 'assistant', 'content': "As an AI, I don't have personal experiences, but I'm here and ready to assist you with any questions or tasks you have! How can I help you today?"},
        {'role': 'user', 'content': "I am unsure"},
        {'role': 'assistant', 'content': "It's okay to feel unsure sometimes. If there's anything specific on your mind or any decision you're facing, feel free to share, and I'll do my best to assist you. Whether it's about making a choice, seeking advice, or just discussing your thoughts, I'm here to help."},
        {'role': 'user', 'content': "yes talk to me"},
        {'role': 'assistant', 'content': "Of course! Is there anything particular on your mind that you'd like to talk about? It could be anything from personal concerns, questions about a topic, or just casual conversation. Feel free to share, and I'm here to listen and offer assistance or insights as best as I can."},
        {'role': 'user', 'content': "i don't know"},
        {'role': 'assistant', 'content': "That's completely fine. Sometimes it can be helpful to explore different topics or interests to see if anything sparks your curiosity or engages your thoughts. Is there a subject you've been curious about or something you've been meaning to learn more about? Alternatively, we can engage in a casual conversation about anything you'd like. Let me know if there's a specific direction you'd like to take our conversation or if there's anything specific you'd like to discuss."},
    ],
]

### Helper function for request - response

In [91]:
def generate_messages(phrase_request: str) -> list[dict]:
    """Using random this function will generate messages to pass to ChatGPT

    Args:
        phrase_request (str): Request phrase to put to the end of messages list

    Returns:
        list[dict]: messages list in a OpenAi format
    """
    # this is a standard option
    return_messages = [{'role': 'system', 'content': "You are a helpful assistant."},
                       ]

    # use random to chose if we are adding anything
    if random.random() < USE_STARTUP_DIALOG:
        # choose number of adding, based on provided weights
        startup_len_to_add = random.choices(
            DIALOG_LENGTH_OPTIONS, weights=DIALOG_LENGTH_WEIGHTS)[0]
        message_chain = random.randint(0, len(starting_messages)-1)
        return_messages += starting_messages[message_chain][:startup_len_to_add]

    # finally our phrase
    return return_messages+[
        {'role': 'user', 'content': phrase_request},
    ]

In [92]:
def request_gpt(messages: list[dict], temperature=TEMPERATURE, model=GPT_MODEL) -> str:
    """Sends messages to ChatGPT and returns the response

    Args:
        messages (list[dict]): list of messages in OpenAI format

    Returns:
        str: response of ChatGpt
    """
    chat_completion = client.chat.completions.create(
        messages=messages,
        model=model,
        temperature=temperature
    )

    return chat_completion.choices[0].message.content

In [98]:
def speak_with_gpt(phrase_request: str, temperature=SPEAK_TEMPERATURE) -> str:
    """Use phrase_request to generate messages with generate_messages function, send's them to ChatGPT, and return response

    Args:
        phrase_request (str): Phrase to ask

    Returns:
        str: Response of ChatGPT
    """
    return request_gpt(generate_messages(phrase_request=phrase_request), temperature=temperature)

### Helper function for comparing the results

In [24]:
# lower is better in such cases, we will get more sustainable responses
COMPARING_TEMPERATURE = 0.3

#
# note: i have made experiments with it until i get it working correctly
#

COMPARING_PROMPT_TEMPLATE_2 = """
You will need to compare two phrases and scale their likeness. 

Compare PHRASE_1 and PHRASE_2 by summarized meaning, rate similarity by scale from 0 to 100 (0 - no similarity or opposite meaning, 100 - total likeness, similar meaning).

Response with number only!


```PHRASE_1```
<<<PHRASE1###>>>
```END OF PHRASE_1```


```PHRASE_2```
<<<PHRASE2###>>>
```END OF PHRASE_2```

SCALE OF LIKENESS:
"""

In [25]:
def generate_comparing_prompt(response1: str, response2: str, comparing_template=COMPARING_PROMPT_TEMPLATE_2) -> str:
    """Insert into COMPARING_PROMPT_TEMPLATE given request and responses

    Args:
        response1 (str): response1
        response2 (str): response2

    Returns:
        str: Prompt to send to ChatGRT for comparing
    """
    return comparing_template.replace('<<<PHRASE1###>>>', response1).replace('<<<PHRASE2###>>>', response2)

In [26]:
def compare_with_gpt(response1: str, response2: str, temperature=COMPARING_TEMPERATURE, model=GPT_MODEL) -> int:
    """Makes a prompt to compare phrase1 and phrase2, send to ChatGPT and return and integer 

    Args:
        response1 (str): response1
        response2 (str): response2

    Returns:
        int: 0..100 - scale of likeness, -1 if something goes wrong
    """
    messages = [{'role': 'system', 'content': "You are a helpful assistant. "},
                {'role': 'user', 'content': generate_comparing_prompt(
                    response1, response2)},
                ]
    response = request_gpt(messages, temperature=temperature)

    try:
        scale_number = int(response.strip())
    except ValueError:
        scale_number = -1

    return scale_number

### Labeling

we will label the requests for futher analysis

In [27]:
LABELING_TEMPERATURE = 0

LABEL_PROMPT_1 = """
Analyze the text and label the topic of it. Label must be of one or several words, and name the type of mental disorder the author of text is facing. 
Examples of topics: [ Depression, Anxiety, Eating Disorder, Drug Use, Addiction, Schizophrenia, Bipolar Disorder, Self-Esteem, Personality Disorders, .... ]
You should use common label names with low variety, so I could use this label later in my project.
Provide two labels maximum, that are more relevant.
Respond with label names only, separate different labels with coma ","!
<<<TEXT>>>
<<<<<<TEXT###>>>>>>
<<<END OF TEXT>>>

<<<LABEL>>>:
"""


def generate_labeling_prompt(text_to_label: str, comparing_template=LABEL_PROMPT_1) -> str:
    """Insert into comparing_template given text to send later to ChatGPT

    Args:
        text_to_label (str): text to label

    Returns:
        str: Prompt to send to ChatGRT for labeling
    """
    return comparing_template.replace('<<<<<<TEXT###>>>>>>', text_to_label)


def get_label_from_chatGPT(text_to_label: str, temperature=LABELING_TEMPERATURE) -> str:
    """Generate label prompt, send to ChatGPT and get label for given text

    Args:
        text_to_label (str): text to label

    Returns:
        str: label
    """
    messages = [{'role': 'system', 'content': "You are a helpful assistant. You label text with labels from a limited common set, so they could later be used to aggregate texts."},
                {'role': 'user', 'content': generate_labeling_prompt(
                    text_to_label)},
                ]
    response = request_gpt(messages, temperature=temperature)

    return response

## Simulation 

We will make a new dataset and run a simulation for filling it 

In [28]:

gpt_simulation = pd.DataFrame(columns=['request', 'label', 'ds_responses_count',
                              'chatgpt_response', 'scores', 'min_score', 'max_score', 'mean_score'])

GPT_SIMULATION_DATASET_FILENAME = 'simulation_data/gpt_simulation_01_'

In [109]:
def run_one_request(request:str,label:str,gpt_dataset=gpt_simulation,max_comparing=10):
    """Send's request_str to ChatGPT and analyze the response.

    Args:
        request_str (str): Request to send to chatGPT
        label (str): Label of request
        dataset (_type_, optional): Dataset to write results. Defaults to gpt_simulation.
        max_comparing (int, optional): To how many original dataset responses we should compare this GPTresponse. Defaults to 15.
        
    """
    current_ds_len=len(gpt_dataset)
    ds_responses_count=context_unique[context_unique['context']==request]['count'].iloc[0]
    ds_responses=conversation_dataset[conversation_dataset['context']==request]['response']
    if len(ds_responses)!=ds_responses_count:
        print("ERROR WITH COUNT MISMATCH")
        return
    print('SIM ('+str(current_ds_len)+') ['+request[:20]+'] ', end='')
    chatgpt_response=speak_with_gpt(request, temperature=SPEAK_TEMPERATURE)
    print(' -> ','{'+chatgpt_response[:100]+'} :: [', end='')
    scores=[]
    for i in range(min(ds_responses_count, max_comparing)):
        one_score=compare_with_gpt(chatgpt_response,ds_responses.iloc[i])
        scores.append(one_score)
        print(one_score,' ', end='')
    scores=np.array(scores)
    min_score, max_score, mean_score=scores.min(), scores.max(),scores.mean()
    print(f' ]  min: {min_score},  max: {max_score},  mean: {mean_score} ')
    gpt_dataset.loc[current_ds_len]={
                'request': request,
                'label': label,
                'ds_responses_count': ds_responses_count,         
                'chatgpt_response': chatgpt_response,
                'scores': scores,
                'min_score': min_score,
                'max_score': max_score,
                'mean_score': mean_score
    }
    

In [29]:
req_num=12

sample_request=context_unique.iloc[req_num]['context']

sample_request

'After I told them, they yelled at me.'

In [32]:
label=get_label_from_chatGPT(sample_request)

run_one_request(sample_request, label)

SIM (0) [After I told them, they yelled at me.] (L: Family Conflict, Anxiety ) ->  {I'm sorry to hear that. It sounds like a difficult} :: [70  70  70  70  75  70  75  20   ]  min: 20,  max: 75,  mean: 65.0 


In [33]:
req_num=17

sample_request=context_unique.iloc[req_num]['context']

sample_request

"After my fiancé cheated on me because of a miscommunication, I pulled myself away from him.    Now he says all he wants is be with me. I'm pushing him away mentally now because he cheated on me again. I make plans with him that I don't keep. I really do love him still and I care about him. Should I give home one more chance?"

In [34]:
label=get_label_from_chatGPT(sample_request)

run_one_request(sample_request, label)

SIM (1) [After my fiancé cheated on me because of a miscomm] (L: Relationship Issues, Trust Issues ) ->  {I'm sorry to hear that you're going through this d} :: [63   ]  min: 63,  max: 63,  mean: 63.0 


In [35]:
len(gpt_simulation)

2

In [36]:
gpt_simulation

,request,label,ds_responses_count,chatgpt_response,scores,min_score,max_score,mean_score
0,"After I told them, they yelled at me.","Family Conflict, Anxiety",8,I'm sorry to hear that. It sounds like a diffi...,"[70, 70, 70, 70, 75, 70, 75, 20]",20,75,65.0
1,After my fiancé cheated on me because of a mis...,"Relationship Issues, Trust Issues",1,I'm sorry to hear that you're going through th...,[63],63,63,63.0


In [39]:
def save_simulation_dataset(gpt_dataset=gpt_simulation):
    """Saves to csv the simulation dataset, using INDEX and Dataset lenght to make the filename

    Args:
        dataset (_type_, optional): _description_. Defaults to gpt_simulation.
    """
    ds_len = len(gpt_dataset)
    print()
    if ds_len:
        filename=GPT_SIMULATION_DATASET_FILENAME+str(ds_len).zfill(4)+'.csv'
        print(ds_len,' len dataset will be saved to ',filename)
        gpt_dataset.to_csv(filename)
        print('Dataset saved to ', filename)

In [40]:
save_simulation_dataset()


2  len dataset will be saved to  simulation_data/gpt_simulation_01_0002.csv
Dataset saved to  simulation_data/gpt_simulation_01_0002.csv


In [41]:
req_num=22

sample_request=context_unique.iloc[req_num]['context']

sample_request

"Any time things get heated, the past gets brought up. I raise my voice at her to try to get a point across. She asks open-ended questions but always has to turn my own answer against me. She's seven months pregnant. I have drank in the past, but I'm not dependent on alcohol—I only drink at social events."

In [42]:
label=get_label_from_chatGPT(sample_request)

run_one_request(sample_request, label)

SIM (2) [Any time things get heated, the past gets brought ] (L: Relationship Issues, Alcohol Abuse ) ->  {It sounds like there may be some communication iss} :: [70   ]  min: 70,  max: 70,  mean: 70.0 


In [43]:
req_num=41

sample_request=context_unique.iloc[req_num]['context']

sample_request

'Every winter I find myself getting sad because of the weather. How can I fight this?'

In [44]:
label=get_label_from_chatGPT(sample_request)

run_one_request(sample_request, label)

SIM (3) [Every winter I find myself getting sad because of ] (L: Seasonal Affective Disorder, Depression ) ->  {I'm sorry to hear that you struggle with feeling s} :: [85  75  85  85  90  85  85  85  85  70  70  90  70  85  85   ]  min: 70,  max: 90,  mean: 82.0 


In [45]:
gpt_simulation

,request,label,ds_responses_count,chatgpt_response,scores,min_score,max_score,mean_score
0,"After I told them, they yelled at me.","Family Conflict, Anxiety",8,I'm sorry to hear that. It sounds like a diffi...,"[70, 70, 70, 70, 75, 70, 75, 20]",20,75,65.0
1,After my fiancé cheated on me because of a mis...,"Relationship Issues, Trust Issues",1,I'm sorry to hear that you're going through th...,[63],63,63,63.0
2,"Any time things get heated, the past gets brou...","Relationship Issues, Alcohol Abuse",1,It sounds like there may be some communication...,[70],70,70,70.0
3,Every winter I find myself getting sad because...,"Seasonal Affective Disorder, Depression",15,I'm sorry to hear that you struggle with feeli...,"[85, 75, 85, 85, 90, 85, 85, 85, 85, 70, 70, 9...",70,90,82.0


In [47]:
save_simulation_dataset()


4  len dataset will be saved to  simulation_data/gpt_simulation_01_0004.csv
Dataset saved to  simulation_data/gpt_simulation_01_0004.csv


In [48]:
test=pd.read_csv('simulation_data/gpt_simulation_01_0004.csv')

In [49]:
test

,Unnamed: 0,request,label,ds_responses_count,chatgpt_response,scores,min_score,max_score,mean_score
0,0,"After I told them, they yelled at me.","Family Conflict, Anxiety",8,I'm sorry to hear that. It sounds like a diffi...,[70 70 70 70 75 70 75 20],20,75,65.0
1,1,After my fiancé cheated on me because of a mis...,"Relationship Issues, Trust Issues",1,I'm sorry to hear that you're going through th...,[63],63,63,63.0
2,2,"Any time things get heated, the past gets brou...","Relationship Issues, Alcohol Abuse",1,It sounds like there may be some communication...,[70],70,70,70.0
3,3,Every winter I find myself getting sad because...,"Seasonal Affective Disorder, Depression",15,I'm sorry to hear that you struggle with feeli...,[85 75 85 85 90 85 85 85 85 70 70 90 70 85 85],70,90,82.0


### Ready to go!


Ok, we will start with entries, that have 3 or more original dataset responses
We will be doing 3 trials for each request

In [115]:
del gpt_simulation

gpt_simulation = pd.DataFrame(columns=['request', 'label', 'ds_responses_count',
                              'chatgpt_response', 'scores', 'min_score', 'max_score', 'mean_score'])

GPT_SIMULATION_DATASET_FILENAME = 'simulation_data/gpt_simulation_02_'

In [116]:
NUM_TRIALS=3

MAX_COMPARING=10

SAVE_ON_NUMBER=25

context_from_3_responses=context_unique[context_unique['count']>2]


In [118]:
sim_length=len(context_from_3_responses)

In [111]:
for request_number in range(sim_length):
    request=context_from_3_responses.iloc[request_number].iloc[0]
    label=get_label_from_chatGPT(request)
    print(f'context[{request_number}][{request[:150]}] (L:{label}) :')
                                          
    for trial in range(NUM_TRIALS):
        run_one_request(request,label,gpt_dataset=gpt_simulation,max_comparing=MAX_COMPARING)
    
    if (request_number % SAVE_ON_NUMBER==0)and(request_number>0):
        save_simulation_dataset(gpt_dataset=gpt_simulation)   
        print()
    print()
        

context[0][A few years ago I was making love to my wife when for no known reason I lost my erection,    Now I'm In my early 30s and my problem has become more an] (L:Erectile Dysfunction, Depression) :
SIM (4) [A few years ago I wa]  ->  {I'm sorry to hear that you're going through such a challenging situation. Erectile dysfunction (ED) } :: [72  70  85  20  75  85   ]  min: 20,  max: 85,  mean: 67.83333333333333 
context[1][After 40 years of being straight, how could I now find myself interested in people of the same sex? I have had a few same sex encounters of my own doi] (L:Sexuality, Identity Crisis) :
SIM (5) [After 40 years of be]  ->  {It's completely normal for sexual orientation to be fluid and for individuals to experience changes } :: [85  75  90  90  73  70   ]  min: 70,  max: 90,  mean: 80.5 
context[2][After I told them, they yelled at me.] (L:Family Conflict, Anxiety) :
SIM (6) [After I told them, t]  ->  {I'm sorry to hear that you had to go through that experience. It 

In [117]:
gpt_simulation

,request,label,ds_responses_count,chatgpt_response,scores,min_score,max_score,mean_score
